In [1]:
import unicodedata
import string
import glob
import os
import torch.nn as nn
import torch
import random
import time
import math


In [2]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

def unicodeToAscii(s):
    # remove the accents
    
    return ''.join(c for c in unicodedata.normalize('NFD', s)if unicodedata.category(c) != 'Mn' and c in all_letters)

In [3]:
unicodeToAscii('Ślusàrski')

'Slusarski'

In [4]:
def readLines(filename):

    lines = open(filename, encoding='utf-8').read()
    lines = lines.strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

In [5]:
category_lines = {}
all_categories = []

for filename in glob.glob('data/names/*.txt'):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines
    
n_categories = len(all_categories)

In [6]:
print(all_categories)

['German', 'Polish', 'Irish', 'Korean', 'Chinese', 'Japanese', 'Czech', 'Arabic', 'Greek', 'Scottish', 'Italian', 'French', 'Portuguese', 'Spanish', 'Vietnamese', 'Dutch', 'Russian', 'English']


In [7]:
print(category_lines['Italian'][:5])

['Abandonato', 'Abatangelo', 'Abatantuono', 'Abate', 'Abategiovanni']


In [8]:
# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

In [9]:
print(len(all_letters))
print(letterToTensor('J'))

57
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]])


In [10]:
print(lineToTensor('Jones'))

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0

In [11]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        
        super(RNN, self).__init__()

        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        
        return torch.zeros(1, self.hidden_size)


In [12]:
n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [13]:
input = letterToTensor('A')
hidden =torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

In [14]:
input = lineToTensor('Albert')
hidden = torch.zeros(1, n_hidden)
output, next_hidden = rnn(input[0], hidden)
print(output)

tensor([[-2.7793, -2.8770, -2.9034, -2.9070, -2.8856, -2.9991, -2.8922, -2.9692,
         -2.9130, -2.9269, -2.9882, -2.8469, -2.9284, -2.8362, -2.8608, -2.7863,
         -2.8979, -2.8599]], grad_fn=<LogSoftmaxBackward>)


In [15]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
#     print(top_n)
#     print(top_i)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i

print(categoryFromOutput(output))

('German', 0)


In [16]:
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor

for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)

category = Czech / line = Cermak
category = Arabic / line = Khouri
category = Czech / line = Jirik
category = Vietnamese / line = Vuong
category = Spanish / line = Menendez
category = Chinese / line = Sum
category = Japanese / line = Abe
category = Portuguese / line = Costa
category = Czech / line = Tykal
category = Italian / line = Abate


In [17]:
criterion = nn.NLLLoss()

In [18]:
learning_rate = 0.005 # If you set this too high, it might explode. If too low, it might not learn

def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()
    
    rnn.zero_grad()
    
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
#         import pdb; pdb.set_trace()

    loss = criterion(output, category_tensor)
    loss.backward()
    # Add parameters' gradients to their values, multiplied by learning rate
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)
#         import pdb; pdb.set_trace()
    return output, loss.item()

In [19]:
n_iters = 100000
print_every = 5000
plot_every = 1000

# Keep track of losses for plotting
current_loss = 0
all_losses = []

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and guess
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)
        correct = '✓' if guess == category else '✗ (%s)' % category
        print('%d %d%% (%s) %.4f %s / %s %s' % (iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

5000 5% (0m 7s) 2.4484 Najjar / Arabic ✓
10000 10% (0m 13s) 2.3007 Gerald / Dutch ✗ (Irish)
15000 15% (0m 19s) 2.3056 Behrend / English ✗ (German)
20000 20% (0m 26s) 0.6913 Ghannam / Arabic ✓
25000 25% (0m 33s) 0.4918 Valentini / Italian ✓
30000 30% (0m 40s) 2.0547 Castro / Portuguese ✗ (Spanish)
35000 35% (0m 46s) 2.5070 Stewart / Dutch ✗ (Scottish)
40000 40% (0m 53s) 2.0177 Sherak / Portuguese ✗ (Czech)
45000 45% (0m 59s) 1.6380 Fabron / French ✓
50000 50% (1m 6s) 0.7163 Cha / Korean ✓
55000 55% (1m 13s) 4.5362 Macdermott / German ✗ (Irish)
60000 60% (1m 20s) 3.2287 Saitoh / Arabic ✗ (Japanese)
65000 65% (1m 27s) 1.4910 Pinho / Korean ✗ (Portuguese)
70000 70% (1m 34s) 0.3834 Wyrick / Polish ✓
75000 75% (1m 41s) 0.6538 Hwang / Korean ✓
80000 80% (1m 47s) 1.4802 Park  / Polish ✗ (Korean)
85000 85% (1m 54s) 0.5419 Basara / Arabic ✓
90000 90% (2m 0s) 0.5720 Romao / Portuguese ✓
95000 95% (2m 7s) 0.2344 Palmeiro / Portuguese ✓
100000 100% (2m 14s) 3.7243 Holguin / English ✗ (Spanish)


In [20]:
plt.figure()
plt.plot(all_losses)

NameError: name 'plt' is not defined

In [ ]:
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output

def predict(input_line, n_predictions=3):
    print('\n> %s' % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print('(%.2f) %s' % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])

predict('Dovesky')
predict('Jackson')
predict('Satoshi')